# Analyse d'images en temps réel avec Azure et OpenCV

Ce script capture des images à partir de la caméra, les analyse en utilisant le service Azure Cognitive Services pour détecter des objets et générer des légendes, puis affiche les résultats en temps réel.

# Importation des bibliothèques

In [1]:
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import cv2
from time import sleep

Ces lignes importent les bibliothèques nécessaires pour l'analyse d'images avec Azure, la manipulation de la vidéo avec OpenCV, et l'affichage des images dans un notebook Jupyter.

# Configuration du client Azure

In [2]:
# Dinition de l'endpoint et la clé d'API pour le service Azure Cognitive Services.
endpoint = "https://votre-endpoint.cognitiveservices.azure.com/"
key = "votre-cle-d-api"

# Création d'une instance du client ImageAnalysisClient avec les informations d'identification.
client = ImageAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

Ici, nous configurons le client Azure pour l'analyse d'images en utilisant l'endpoint et la clé d'API fournis.

# Initialisation de la capture vidéo


In [3]:
# Initialisation de la capture vidéo à partir de la caméra par défaut.
cam = cv2.VideoCapture(0)

# Obtention les dimensions du cadre vidéo.
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

Ces lignes initialisent la capture vidéo à partir de la caméra par défaut et obtiennent les dimensions du cadre vidéo.

# Boucle principale pour la capture et l'analyse des images

Cette boucle capture chaque image de la vidéo, l'encode en JPEG, et envoie les données d'image au service Azure pour analyse.


In [4]:
while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Encodez l'image capturée en JPEG.
    _, buffer = cv2.imencode('.jpg', frame)
    image_data = buffer.tobytes()

    # Analysez l'image en utilisant le service Azure.
    result = client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.OBJECTS, VisualFeatures.CAPTION]
    )

    ## Dessin des objets détectés et affichage des résultats
    # Dessin des rectangles autour des objets détectés et affichage de leurs noms.
    for object in result["objectsResult"]["values"]:
        x1, y1 = object["boundingBox"]['x'], object["boundingBox"]['y']
        x2, y2 = x1 + object["boundingBox"]["w"], y1 + object["boundingBox"]["h"]
        name = object["tags"][0]["name"]
        confidence = object["tags"][0]["confidence"]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
        cv2.putText(frame, name, (x1, y1), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)

    # Affichez la légende générée par le service Azure.
    caption = result["captionResult"]
    caption_text = caption["text"]
    x = int(frame_width/2 - len(caption_text)*4)
    y = 15
    cv2.putText(frame, caption_text, (x, y), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255))

    # Affichage de l'image avec la reconnaissance d'objets AI
    cv2.imshow('Camera with AI Object Recognition', frame)

    # Arrêter la boucle si la touche 'c' est pressée.
    if cv2.waitKey(1)==ord('c'):
       break

    # Attendre 3 secondes avant de capturer la prochaine image. (Dû à la limitation de l'API gratuite)
    sleep(3)

# Libérez la caméra et fermez toutes les fenêtres OpenCV.
cam.release()
cv2.destroyAllWindows()

ServiceRequestError: <urllib3.connection.HTTPSConnection object at 0x00000212192A80B0>: Failed to resolve 'votre-endpoint.cognitiveservices.azure.com' ([Errno 11001] getaddrinfo failed)

Cette section affiche la vidéo annotée. La boucle continue à capturer et analyser les images toutes les 3 secondes.

